In [2]:
import pandas as pd
import numpy as np
import gpflow as gpf

In [4]:
#import data: 
dat = pd.read_csv('./GP_365_Preprocessed.csv', index_col = 'Unnamed: 0')
print(dat.shape)
dat.head()

(151609, 5)


,site,MonitorData,day,x,y
0,0,5.9,15,3.767606e+06,739558.671618
1,0,11.0,18,3.767606e+06,739558.671618
2,0,8.6,21,3.767606e+06,739558.671618
3,0,6.2,24,3.767606e+06,739558.671618
4,0,7.6,27,3.767606e+06,739558.671618


In [14]:
#split into X and Y 

#X is [time, xloc, yloc]
X_dat = dat.loc[:, ['day', 'x', 'y']].values 

#Y is Monitor Values
Y_dat = dat.MonitorData.values.reshape(Y_dat.size, 1)

In [24]:
#Make simpe GP model
with gpf.defer_build():
    X = X_dat
    Y = Y_dat
    m = gpf.models.GPR(X, Y, kern=gpf.kernels.RBF(3))

In [25]:
#Review GP model 
m.as_pandas_table()

,class,prior,transform,trainable,shape,fixed_shape,value
GPR/kern/variance,Parameter,None,+ve,True,(),True,1.0
GPR/kern/lengthscales,Parameter,None,+ve,True,(),True,1.0
GPR/likelihood/variance,Parameter,None,+ve,True,(),True,1.0


In [26]:
#Set gamma prior on RBF kernel params: 
m.kern.variance.prior = gpf.priors.Gamma(2,3)
m.kern.lengthscales.prior = gpf.priors.Gamma(2,3)
m.as_pandas_table()

,class,prior,transform,trainable,shape,fixed_shape,value
GPR/kern/variance,Parameter,"Ga([ 2.],[ 3.])",+ve,True,(),True,1.0
GPR/kern/lengthscales,Parameter,"Ga([ 2.],[ 3.])",+ve,True,(),True,1.0
GPR/likelihood/variance,Parameter,None,+ve,True,(),True,1.0


In [27]:
#Train params 
m.compile()
opt = gpf.train.ScipyOptimizer()
opt.minimize(m)

AttributeError: 'Variable' object has no attribute 'shape'

In [28]:
print(type(X))

<class 'numpy.ndarray'>
